<a href="https://colab.research.google.com/github/abdalrahmanmajzoub/Cryptocurrancy-changes-using-coingecko/blob/main/cryptocurrency_changes_using_coingecko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance

     |████████████████████████████████| 62 kB 895 kB/s 


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

Lets start off by getting the data we need. Using the get_coin_market_chart_by_id(id, vs_currency, days). id is the name of the coin you want, vs_currency is the currency you want the price in, and days is how many days back from today you want.

In [3]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

In [4]:
type(bitcoin_data )
#check the keys 
bitcoin_data.keys()

dict_keys(['prices', 'market_caps', 'total_volumes'])

In [5]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:10]

[[1629381640189, 44973.25287391277],
 [1629385274051, 45363.27399065973],
 [1629388955177, 45786.215355430104],
 [1629392553757, 45720.611100250244],
 [1629396105595, 45749.64350861408],
 [1629399608391, 46639.298229741646],
 [1629403239685, 46741.30663234053],
 [1629406859081, 46628.068267601084],
 [1629410452238, 46642.53156050792],
 [1629414050080, 46560.56155934897]]

lets turn this data into a Pandas DataFrame.

In [6]:
data  = pd.DataFrame(bitcoin_price_data,columns=['TimeStamp','Price'])

In [7]:
data.head()

,TimeStamp,Price
0,1629381640189,44973.252874
1,1629385274051,45363.273991
2,1629388955177,45786.215355
3,1629392553757,45720.611100
4,1629396105595,45749.643509


Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called Date. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [8]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

In [9]:
data.head()

,TimeStamp,Price,date
0,1629381640189,44973.252874,2021-08-19
1,1629385274051,45363.273991,2021-08-19
2,1629388955177,45786.215355,2021-08-19
3,1629392553757,45720.611100,2021-08-19
4,1629396105595,45749.643509,2021-08-19


Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [10]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

In [11]:
candlestick_data.shape

(31, 5)

In [12]:
candlestick_data.head()

date         Price                                          
                        min           max         first          last
0  2021-08-19  44973.252874  46741.306632  44973.252874  46560.561559
1  2021-08-20  46745.034623  49124.360559  46745.034623  49124.360559
2  2021-08-21  48572.974423  49658.887046  49195.878602  49364.850589
3  2021-08-22  48377.821512  49253.161690  48867.876579  49117.657045
4  2021-08-23  49237.272831  50311.818126  49237.272831  49621.937622

Finally we are now ready to use plotly to create our Candlestick Chart.

In [13]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()